In [3]:
# Playoffs Round/Series EV Calculator (Isolated to the Matchup)


import mysql.connector
from collections import defaultdict

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conf, Champ
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch results and organize them in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# Define the relevant teams by full names
relevant_teams = [
    
    ## INSERT FULL TEAM NAMES HERE
    'Detroit Tigers', 'Cleveland Guardians'
]

# Fetch and organize the payouts by relevant teams
for participant_name, event_type, total_payout in cursor.fetchall():
    # Check if the participant name matches any relevant team
    if participant_name in relevant_teams:
        team_name = participant_name
        total_payout = float(total_payout)  # Convert to float for compatibility
        
        if event_type == 'Conference Winner':
            payouts[team_name]['payout_conference'] = total_payout
        elif event_type == 'Championship':
            payouts[team_name]['payout_championship'] = total_payout
        
        # Print out the potential payout for verification
        print(f"Team: {team_name}, Event Type: {event_type}, Total Payout: {total_payout}")

cursor.close()
conn.close()

# Define the EV calculation functions
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Inputs for the two teams
team1 = { 
    
    ## INSERT PROBABILITIES BELOW
    
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": payouts['Detroit Tigers']['payout_conference'],             
    "champ_payout": payouts['Detroit Tigers']['payout_championship']               
}

team2 = { 
    "current_round_prob": 1 - team1["current_round_prob"],      
    
     ## INSERT PROBABILITIES BELOW
    
    "current_conference_prob": 0.15,    
    "current_champ_prob": 0.06,       
    "conference_payout": payouts['Cleveland Guardians']['payout_conference'],  
    "champ_payout": payouts['Cleveland Guardians']['payout_championship']   
}

# Calculate conditional EVs if each team wins the current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# Calculate the Series Delta as the difference between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

# Output results
print("Conditional Expected Values if Team Wins Current Round:")
print(f"Detroit Tigers EV if win round: ${team1_ev_win:.2f}")
print(f"Cleveland Guardians EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"${series_delta:.2f}")


Team: Cleveland Guardians, Event Type: Championship, Total Payout: 59160.0
Team: Detroit Tigers, Event Type: Championship, Total Payout: 204452.0
Conditional Expected Values if Team Wins Current Round:
Detroit Tigers EV if win round: $22972.13
Cleveland Guardians EV if win round: $7617.17

Series Delta:
$15354.97


In [4]:
## Playoffs Individual Game EV Calculator (Isolated to the Matchup)


def calculate_team_ev(new_series_odds, series_ev):
    """Calculate the team EV based on new series odds and series EV."""
    return new_series_odds * series_ev

def calculate_game3_ev_delta(team1_game3_win_prob, team2_game3_win_prob, team1_new_series_odds, team2_new_series_odds, team1_ev_win, team2_ev_win):
    """Calculate the EV Delta for Game 3."""
    
    # Calculate EV if Team 1 wins Game 3
    team1_ev_if_win = calculate_team_ev(team1_new_series_odds, team1_ev_win)
    team2_ev_if_lose = calculate_team_ev(1 - team1_new_series_odds, team2_ev_win)
    ev_if_team1_wins_game3 = team1_ev_if_win + team2_ev_if_lose
    
    # Calculate EV if Team 2 wins Game 3
    team1_ev_if_lose = calculate_team_ev(1 - team2_new_series_odds, team1_ev_win)
    team2_ev_if_win = calculate_team_ev(team2_new_series_odds, team2_ev_win)
    ev_if_team2_wins_game3 = team1_ev_if_lose + team2_ev_if_win
    
    # Calculate the EV Delta for Game 3
    ev_delta = abs(ev_if_team1_wins_game3 - ev_if_team2_wins_game3)
    
    return ev_if_team1_wins_game3, ev_if_team2_wins_game3, ev_delta

# Inputs specific to Game 3
team1_game3_win_prob = 0.53  # Team 1's probability of winning Game 3
team2_game3_win_prob = (1 - team1_game3_win_prob)  # Team 2's probability of winning Game 3
team1_new_series_odds = 1  # Series odds if Team 1 wins Game 3
team2_new_series_odds = 0.99148936170  # Series odds if Team 2 wins Game 3

# Use team1_ev_win and team2_ev_win directly from the previous script
# Calculate EV if Team 1 or Team 2 wins Game 3 and the EV Delta
ev_if_team1_wins_game3, ev_if_team2_wins_game3, game3_ev_delta = calculate_game3_ev_delta(
    team1_game3_win_prob,
    team2_game3_win_prob,
    team1_new_series_odds,
    team2_new_series_odds,
    team1_ev_win,  # From previous script
    team2_ev_win   # From previous script
)

# Output results
print("Expected Values if either team wins Game 3:")
print(f"EV if Team 1 wins Game 3: ${ev_if_team1_wins_game3:.2f}")
print(f"EV if Team 2 wins Game 3: ${ev_if_team2_wins_game3:.2f}")
print("\nGame 3 EV Delta:")
print(f"EV Delta: ${game3_ev_delta:.2f}")


Expected Values if either team wins Game 3:
EV if Team 1 wins Game 3: $22972.13
EV if Team 2 wins Game 3: $7747.85

Game 3 EV Delta:
EV Delta: $15224.29
